In [1]:
import polars as pl
import polars.selectors as cs

import numpy as np
import matplotlib.pyplot as plt

from catboost import Pool, CatBoostClassifier
from sklearn.metrics import roc_auc_score

from pycaret.classification import ClassificationExperiment

In [2]:
data_path = '/media/dm/1TB/Data Fusion Contest 2026/task2/'

In [3]:
%%time
train = pl.scan_parquet(data_path+'train_main_features.parquet'
         #).join(pl.scan_parquet(data_path+'train_extra_features.parquet'),
         #       on="customer_id"
                          ).with_columns(cs.by_dtype(pl.Float64).cast(pl.Float32)#.fill_null(-99999)
                          ).with_columns(cs.starts_with("cat_feature_").cast(pl.Int32)#.fill_null(-1)#.cast(pl.Categorical)
                                        ).collect()
train

CPU times: user 986 ms, sys: 1.72 s, total: 2.71 s
Wall time: 497 ms


customer_id,cat_feature_1,cat_feature_2,cat_feature_3,cat_feature_4,cat_feature_5,cat_feature_6,cat_feature_7,cat_feature_8,cat_feature_9,cat_feature_10,cat_feature_11,cat_feature_12,cat_feature_13,cat_feature_14,cat_feature_15,cat_feature_16,cat_feature_17,cat_feature_18,cat_feature_19,cat_feature_20,cat_feature_21,cat_feature_22,cat_feature_23,cat_feature_24,cat_feature_25,cat_feature_26,cat_feature_27,cat_feature_28,cat_feature_29,cat_feature_30,cat_feature_31,cat_feature_32,cat_feature_33,cat_feature_34,cat_feature_35,cat_feature_36,…,num_feature_96,num_feature_97,num_feature_98,num_feature_99,num_feature_100,num_feature_101,num_feature_102,num_feature_103,num_feature_104,num_feature_105,num_feature_106,num_feature_107,num_feature_108,num_feature_109,num_feature_110,num_feature_111,num_feature_112,num_feature_113,num_feature_114,num_feature_115,num_feature_116,num_feature_117,num_feature_118,num_feature_119,num_feature_120,num_feature_121,num_feature_122,num_feature_123,num_feature_124,num_feature_125,num_feature_126,num_feature_127,num_feature_128,num_feature_129,num_feature_130,num_feature_131,num_feature_132
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
1000001,1,0,2,1,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,0,0,0,2,0,0,0,1,2,1,2,2,0,0,212,0,0,…,-0.284519,null,-0.004499,null,-0.050159,-0.002297,-0.043592,-0.061488,0.450342,null,0.0,null,-0.060492,-0.279105,null,-0.429813,null,-0.009654,-0.293036,null,-0.493959,-0.019079,null,null,null,null,-0.001357,-0.031281,-0.046146,null,-0.445279,null,null,-0.107666,-0.418616,null,null
1000002,1,0,0,1,0,3,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,212,1,0,…,-0.284519,0.460275,-0.004278,null,-0.556244,-0.002297,-0.043592,-0.061488,1.998252,-0.009552,0.0,-0.005762,-0.060492,-0.279105,null,-0.429813,null,-0.009654,-0.293036,-0.004421,-0.256445,-0.014154,null,-0.24167,null,null,-0.001357,-0.031281,-0.046146,-0.10217,1.550722,null,null,-0.170724,-0.805771,-0.397803,-0.373734
1000003,1,0,0,1,0,3,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,212,0,0,…,-0.284519,-0.264397,-0.004278,null,null,-0.002297,-0.043592,-0.061488,-0.264078,null,0.0,-0.26649,-0.060492,-0.279105,null,-0.429813,null,-0.009654,-0.293036,-0.004421,-0.57313,-0.019124,null,-0.24167,null,null,-0.001357,-0.031281,-0.046146,null,-0.475778,null,null,-0.170724,-0.602005,-0.397803,-0.373734
1000004,1,0,2,1,2,3,2,2,3,2,0,0,0,1,2,2,2,2,2,0,0,2,2,0,0,0,0,2,1,2,2,0,0,212,0,0,…,-0.284519,null,-0.004499,null,null,-0.002297,-0.043592,-0.061488,0.688482,-0.009552,0.0,null,-0.060492,null,null,-0.429813,null,-0.009654,-0.293036,null,-0.57313,null,null,-0.505441,null,0.714631,-0.001357,-0.031281,-0.046146,null,-0.475778,0.111196,0.116695,null,-0.724265,null,null
1000005,1,2,0,1,0,3,0,0,2,1,2,2,2,0,1,0,0,0,0,2,2,2,0,2,2,2,2,0,0,0,0,2,2,212,2,2,…,-0.284519,-0.264397,-0.004499,null,null,-0.002297,-0.043592,null,-0.264078,null,0.0,-0.26649,null,-0.279105,null,-0.429813,null,-0.009654,-0.293036,-0.004421,-0.57313,-0.018674,null,null,null,null,null,null,-0.046146,null,null,null,null,-0.107666,null,-0.397803,-0.373734
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1749996,1,0,2,0,2,3,2,2,4,2,0,0,0,1,2,0,2,2,2,0,0,2,2,0,0,0,0,2,1,2,2,0,0,212,0,0,…,null,null,-0.004499,-0.150243,null,null,null,2.831928,null,null,null,null,-0.060492,null,null,null,null,null,null,null,null,-0.019236,null,-0.505441,null,null,-0.001357,-0.031281,null,null,-0.475778,null,null,null,0.335315,null,null
1749997,1,0,2,1,2,3,2,2,2,2,0,0,0,1,2,0,2,2,2,0,0,0,2,0,0,0,0,2,1,2,2,0,0,212,1,0,…,-0.284519,null,-0.004499,null,-0.510019,-0.002297,-0.043592,-0.061488,-0.264078,-0.009552,0.0,null,-0.060492,-0.279105,null,2.155184,n

In [4]:
target = pl.read_parquet(data_path+'train_target.parquet'
                        ).with_columns(cs.starts_with("target_").cast(pl.Boolean)
                                      )
target

customer_id,target_1_1,target_1_2,target_1_3,target_1_4,target_1_5,target_2_1,target_2_2,target_2_3,target_2_4,target_2_5,target_2_6,target_2_7,target_2_8,target_3_1,target_3_2,target_3_3,target_3_4,target_3_5,target_4_1,target_5_1,target_5_2,target_6_1,target_6_2,target_6_3,target_6_4,target_6_5,target_7_1,target_7_2,target_7_3,target_8_1,target_8_2,target_8_3,target_9_1,target_9_2,target_9_3,target_9_4,target_9_5,target_9_6,target_9_7,target_9_8,target_10_1
i32,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
1000001,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false
1000002,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false
1000003,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true
1000004,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,false,false
1000005,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1749996,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,false,false
1749997,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true,false,false
1749998,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,true


In [5]:
cat_feature_names = [col_name for col_name in train.columns if col_name.startswith("cat_feature")]
#cat_feature_names

In [6]:
%%time

limit = 100000 #если компьютер шустрый ставь 750_000 
minfrac = 0.1 #Желаемая доля в выборке 
s = ClassificationExperiment()
stats=[]

for target_name in target.columns[1:]:
    print("="*60,target_name)
    
    data = train.sample(limit, shuffle = True, seed = 7
                    ).join(target.select(["customer_id",target_name]),
                              on="customer_id"
                             )#.to_pandas()
    if data[target_name].sum()/limit<minfrac:
        data1 = target.select(["customer_id",target_name]
                             ).filter(pl.col(target_name)>0
                                     ).sample( min(round(limit*minfrac), target[target_name].sum()), 
                                              shuffle = True, seed = 7 
                                             )
        data0 = target.select(["customer_id",target_name]
                             ).filter(pl.col(target_name)==0
                                     ).sample( limit-data1.shape[0], 
                                              shuffle = True, seed = 7 
                                             )
        print(data[target_name].sum(), round(limit*minfrac), data1.shape[0], data0.shape[0])
        data = train.sample(limit, 
                             shuffle = True, 
                             seed = 7
                            ).join(pl.concat([data0, data1]),
                                      on="customer_id"
                                     ).drop("customer_id")
    data = data.to_pandas()
    
    #заплатка на глюк polars при переносе категориальных колонок в pandas 
    data[cat_feature_names] = data[cat_feature_names].astype("str")
    
    try:
        s.setup(data, 
            target = target_name, 
            log_experiment=False, 
            experiment_name=target_name
            #use_gpu=True,
           )
        best = s.compare_models(sort = 'AUC')

        leaderboard = s.pull()
        leaderboard.to_csv(f'leaderboard_{target_name}_{limit=}_{minfrac=}')
        stats.append([target_name, 
                      leaderboard.iloc[0].Model,leaderboard.iloc[0].AUC, 
                      leaderboard.iloc[1].Model,leaderboard.iloc[1].AUC, 
                      leaderboard.iloc[2].Model,leaderboard.iloc[2].AUC
                     ]
                    )
        #print(stats)

        s.save_model(best, f'caret_{target_name}_{limit=}_{minfrac=}')
        #break
    except:
        print("Ашипка.")
    
stats = pl.DataFrame(stats, schema=["tcol","model","AUC","model2","AUC2","model3","AUC3"])
stats

============================================================ target_1_1
107 2000 2000 8000


,Description,Value
0,Session id,4224
1,Target,target_1_1
2,Target type,Binary
3,Original data shape,"(190, 200)"
4,Transformed data shape,"(190, 275)"
5,Transformed train set shape,"(133, 275)"
6,Transformed test set shape,"(57, 275)"
7,Numeric features,132
8,Categorical features,67
9,Rows with missing values,100.0%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8495,0.8027,0.0000,0.0000,0.0000,-0.0114,-0.0123,4.8040
lightgbm,Light Gradient Boosting Machine,0.8648,0.7943,0.2000,0.2500,0.2000,0.1771,0.1908,2.5030
rf,Random Forest Classifier,0.8495,0.7752,0.0000,0.0000,0.0000,-0.0114,-0.0123,0.3720
xgboost,Extreme Gradient Boosting,0.8566,0.7523,0.2500,0.3500,0.2667,0.2174,0.2341,0.3450
et,Extra Trees Classifier,0.8571,0.7449,0.0500,0.1000,0.0667,0.0514,0.0554,0.3500
gbc,Gradient Boosting Classifier,0.8341,0.6936,0.1000,0.1000,0.1000,0.0543,0.0508,0.3840
ada,Ada Boost Classifier,0.8121,0.6765,0.0500,0.1000,0.0667,-0.0076,-0.0061,0.3480
lr,Logistic Regression,0.8352,0.6727,0.2000,0.1500,0.1700,0.1103,0.1111,0.7660
svm,SVM - Linear Kernel,0.7440,0.6500,0.2000,0.1833,0.1900,0.0466,0.0426,0.3220
ridge,Ridge Classifier,0.7588,0.6121,0.2500,0.1350,0.1690,0.0496,0.0581,0.2970


Transformation Pipeline and Model Successfully Saved
============================================================ target_1_2
30 2000 2000 8000


,Description,Value
0,Session id,2756
1,Target,target_1_2
2,Target type,Binary
3,Original data shape,"(235, 200)"
4,Transformed data shape,"(235, 284)"
5,Transformed train set shape,"(164, 284)"
6,Transformed test set shape,"(71, 284)"
7,Numeric features,132
8,Categorical features,67
9,Rows with missing values,100.0%


...

Transformation Pipeline and Model Successfully Saved
============================================================ target_10_1


,Description,Value
0,Session id,6518
1,Target,target_10_1
2,Target type,Binary
3,Original data shape,"(10000, 201)"
4,Transformed data shape,"(10000, 315)"
5,Transformed train set shape,"(7000, 315)"
6,Transformed test set shape,"(3000, 315)"
7,Numeric features,133
8,Categorical features,67
9,Rows with missing values,100.0%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.7221,0.7076,0.2939,0.6180,0.3973,0.2456,0.2747,1.1590
lr,Logistic Regression,0.7129,0.7027,0.2975,0.5792,0.3930,0.2296,0.2515,1.8900
catboost,CatBoost Classifier,0.7149,0.6999,0.3551,0.5710,0.4376,0.2603,0.2736,15.0890
gbc,Gradient Boosting Classifier,0.7099,0.6949,0.3515,0.5572,0.4309,0.2494,0.2614,3.0430
ridge,Ridge Classifier,0.7116,0.6919,0.2911,0.5768,0.3867,0.2241,0.2466,0.6770
lightgbm,Light Gradient Boosting Machine,0.7027,0.6893,0.3693,0.5357,0.4370,0.2443,0.2521,23.0160
ada,Ada Boost Classifier,0.6977,0.6873,0.3912,0.5226,0.4470,0.2450,0.2501,1.2580
lda,Linear Discriminant Analysis,0.7041,0.6841,0.3341,0.5450,0.4139,0.2309,0.2435,0.8470
et,Extra Trees Classifier,0.7107,0.6797,0.2888,0.5736,0.3839,0.2212,0.2434,1.1600
xgboost,Extreme Gradient Boosting,0.6917,0.6737,0.3903,0.5087,0.4415,0.2338,0.2378,1.0290


Transformation Pipeline and Model Successfully Saved
CPU times: user 14min, sys: 47 s, total: 14min 47s
Wall time: 2h 17min 15s


tcol,model,AUC,model2,AUC2,model3,AUC3
str,str,f64,str,f64,str,f64
"""target_1_1""","""CatBoost Classifier""",0.8027,"""Light Gradient Boosting Machin…",0.7943,"""Random Forest Classifier""",0.7752
"""target_1_2""","""Light Gradient Boosting Machin…",0.6362,"""Ada Boost Classifier""",0.5931,"""K Neighbors Classifier""",0.5595
"""target_1_3""","""Random Forest Classifier""",0.8305,"""Extra Trees Classifier""",0.8211,"""CatBoost Classifier""",0.7982
"""target_1_4""","""SVM - Linear Kernel""",0.8096,"""Logistic Regression""",0.7896,"""Extreme Gradient Boosting""",0.6959
"""target_1_5""","""SVM - Linear Kernel""",0.785,"""Ridge Classifier""",0.7746,"""Logistic Regression""",0.76
…,…,…,…,…,…,…
"""target_9_5""","""Light Gradient Boosting Machin…",0.7352,"""CatBoost Classifier""",0.708,"""Logistic Regression""",0.6882
"""target_9_6""","""Random Forest Classifier""",0.6265,"""Logistic Regression""",0.6249,"""CatBoost Classifier""",0.6156
"""target_9_7""","""CatBoost Classifier""",0.8188,"""Extra Trees Classifier""",0.8139,"""Random Forest Classifier""",0.7493


In [7]:
stats.write_csv(f"stats_caret_{limit=}_{minfrac=}.csv")

In [8]:
stats

tcol,model,AUC,model2,AUC2,model3,AUC3
str,str,f64,str,f64,str,f64
"""target_1_1""","""CatBoost Classifier""",0.8027,"""Light Gradient Boosting Machin…",0.7943,"""Random Forest Classifier""",0.7752
"""target_1_2""","""Light Gradient Boosting Machin…",0.6362,"""Ada Boost Classifier""",0.5931,"""K Neighbors Classifier""",0.5595
"""target_1_3""","""Random Forest Classifier""",0.8305,"""Extra Trees Classifier""",0.8211,"""CatBoost Classifier""",0.7982
"""target_1_4""","""SVM - Linear Kernel""",0.8096,"""Logistic Regression""",0.7896,"""Extreme Gradient Boosting""",0.6959
"""target_1_5""","""SVM - Linear Kernel""",0.785,"""Ridge Classifier""",0.7746,"""Logistic Regression""",0.76
…,…,…,…,…,…,…
"""target_9_5""","""Light Gradient Boosting Machin…",0.7352,"""CatBoost Classifier""",0.708,"""Logistic Regression""",0.6882
"""target_9_6""","""Random Forest Classifier""",0.6265,"""Logistic Regression""",0.6249,"""CatBoost Classifier""",0.6156
"""target_9_7""","""CatBoost Classifier""",0.8188,"""Extra Trees Classifier""",0.8139,"""Random Forest Classifier""",0.7493
